<a href="https://colab.research.google.com/github/mzs0153/Impact-of-Data-Freshness-in-Learning/blob/main/CarPositionPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/My Drive/Data_Freshness_Paper

/content/drive/My Drive/Data_Freshness_Paper


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [3]:
csv_path = "MountainCarDataset.csv"

df = pd.read_csv(csv_path)

In [4]:
feature_keys = [
    "Acceleration",
    "Position",
    "Velocity",
]

In [5]:
def normalize(data, train_split):
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std

In [ ]:
split_fraction = 0.715
train_split = int(split_fraction * int(df.shape[0]))
step = 1
learning_rate = 0.001
batch_size = 256
epochs = 30
Exppoints=20
Age=np.arange(Exppoints)
score=np.zeros(len(Age))
score1=np.zeros(len(Age))
for i in range(len(Age)):
  print(Age[i])
  L=1
  past=L
  future=Age[i] 
  selected_features = [feature_keys[i] for i in [0, 2]]
  features = df[selected_features]
  
  features = normalize(features.values, train_split)
  features = pd.DataFrame(features)
  
  
  train_data = features.loc[0 : train_split - 1]
  val_data = features.loc[train_split:]    
  start = past + future
  end = start + train_split

  x_train = train_data[[0]].values
  
  y_train = features.iloc[start:end][[1]]
  sequence_length = int(past / step)
  
  dataset_train = keras.preprocessing.timeseries_dataset_from_array(
        x_train,
        y_train,
        sequence_length=sequence_length,
        sampling_rate=step,
        batch_size=batch_size,
    )
  
  x_end = len(val_data) - past - future

  label_start = train_split + past + future

  x_val = val_data.iloc[:x_end][[1]].values
  y_val = features.iloc[label_start:][[0]]

  dataset_val = keras.preprocessing.timeseries_dataset_from_array(
      x_val,
      y_val,
      sequence_length=sequence_length,
      sampling_rate=step,
      batch_size=batch_size,
  )


  for batch in dataset_train.take(1):

    inputs, targets = batch

    

    inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
    lstm_out = keras.layers.LSTM(32)(inputs)
    outputs = keras.layers.Dense(1)(lstm_out)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")

  

    history = model.fit(
      dataset_train,
      epochs=epochs,
     
    )
  #score[i]=model.evaluate(dataset_val)
  score1[i]=history.history['loss'][epochs-1]

plt.plot(Age, score1)
np.savez('trainingloss.npz',data=score1)
#np.savez('trainingloss2_val.npz',data=score1)

0
Epoch 1/30
28/28 [==============================] - 3s 39ms/step - loss: 1.0415
Epoch 2/30
28/28 [==============================] - 1s 39ms/step - loss: 0.9552
Epoch 3/30
28/28 [==============================] - 1s 39ms/step - loss: 0.8799
Epoch 4/30
28/28 [==============================] - 1s 39ms/step - loss: 0.8079
Epoch 5/30
28/28 [==============================] - 1s 39ms/step - loss: 0.7376
Epoch 6/30
28/28 [==============================] - 1s 39ms/step - loss: 0.6723
Epoch 7/30
28/28 [==============================] - 1s 39ms/step - loss: 0.6174
Epoch 8/30
28/28 [==============================] - 1s 40ms/step - loss: 0.5760
Epoch 9/30
28/28 [==============================] - 1s 38ms/step - loss: 0.5472
Epoch 10/30
28/28 [==============================] - 1s 39ms/step - loss: 0.5273
Epoch 11/30
28/28 [==============================] - 1s 39ms/step - loss: 0.5125
Epoch 12/30
28/28 [==============================] - 1s 40ms/step - loss: 0.5003
Epoch 13/30
28/28 [================